In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

In [8]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [10]:
%pip install google-genai==1.7.0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-anthropic 0.2.1 requires langchain-core<0.4.0,>=0.3.1, but you have langchain-core 0.1.52 which is incompatible.
llama-index-embeddings-openai 0.1.11 requires llama-index-core<0.11.0,>=0.10.1, but you have llama-index-core 0.11.0 which is incompatible.
llama-index-indices-managed-llama-cloud 0.1.6 requires llama-index-core<0.11.0,>=0.10.0, but you have llama-index-core 0.11.0 which is incompatible.
llama-index-legacy 0.9.48.post3 requires openai>=1.1.0, but you have openai 0.28.0 which is incompatible.
llama-index-llms-openai 0.1.31 requires llama-index-core<0.11.0,>=0.10.57, but you have llama-index-core 0.11.0 which is incompatible.
llama-index-llms-openai 0.1.31 requires openai<2.0.0,>=1.40.0, but you have openai 0.28.0 which is incompatible.
llama-index-multi-modal-llms-openai 0.1.9 requires llama-in


  Using cached google_genai-1.7.0-py3-none-any.whl.metadata (32 kB)
  Using cached anyio-4.9.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached websockets-15.0.1-cp312-cp312-win_amd64.whl.metadata (7.0 kB)
Using cached google_genai-1.7.0-py3-none-any.whl (144 kB)
Using cached anyio-4.9.0-py3-none-any.whl (100 kB)
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
Using cached websockets-15.0.1-cp312-cp312-win_amd64.whl (176 kB)
  Attempting uninstall: anyio
    Found existing installation: anyio 4.6.0
    Uninstalling anyio-4.6.0:
      Successfully uninstalled anyio-4.6.0
  Attempting uninstall: httpx
    Found existing installation: httpx 0.27.2
    Uninstalling httpx-0.27.2:
      Successfully uninstalled httpx-0.27.2


In [16]:
import faiss
import numpy as np
import json
from google import genai
from google.genai import types
client = genai.Client(api_key=GOOGLE_API_KEY)
from google.genai import types


def get_embeddings(texts, task_type="retrieval_document", batch_size=100):
    all_embeddings = []

    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]
        response = client.models.embed_content(
            model="models/text-embedding-004",
            contents=batch,
            config=types.EmbedContentConfig(task_type=task_type),
        )
        embeddings = [e.values for e in response.embeddings]
        all_embeddings.extend(embeddings)

    return np.array(all_embeddings)



In [17]:
# Load your assessments
with open("shl_assessments_individual.json", "r", encoding="utf-8") as f:
    individual_data = json.load(f)

with open("shl_assessments_pre.json", "r", encoding="utf-8") as f:
    prepackaged_data = json.load(f)

all_data = individual_data + prepackaged_data

# Prepare texts to embed — you can customize this format
documents = [d["name"] + " " + " ".join(d.get("test_types", [])) for d in all_data]

# Generate embeddings
document_embeddings = get_embeddings(documents)


In [18]:
# Create FAISS index
dimension = document_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)

# Add vectors to FAISS
index.add(document_embeddings)

# Save metadata for later retrieval
metadata = all_data


In [19]:
def search_assessments(query_text, k=10):
    query_vector = get_embeddings([query_text], task_type="retrieval_query")
    distances, indices = index.search(query_vector, k)
    return [metadata[i] for i in indices[0]]


In [22]:
results = search_assessments("Frontend developer with JavaScript and UI skills")

for r in results:
    print(r["name"], "-", r["url"],"-", r["remote_support"], "-", r["adaptive_support"], "-", r["test_types"])


ReactJS (New) - https://www.shl.com/solutions/products/product-catalog/view/reactjs-new/ - Yes - No - ['K']
AngularJS (New) - https://www.shl.com/solutions/products/product-catalog/view/angularjs-new/ - Yes - No - ['K']
Salesforce Development (New) - https://www.shl.com/solutions/products/product-catalog/view/salesforce-development-new/ - Yes - No - ['K']
Node.js (New) - https://www.shl.com/solutions/products/product-catalog/view/node-js-new/ - Yes - No - ['K']
HTML/CSS (New) - https://www.shl.com/solutions/products/product-catalog/view/htmlcss-new/ - Yes - No - ['K']
MuleSoft Development (New) - https://www.shl.com/solutions/products/product-catalog/view/mulesoft-development-new/ - Yes - No - ['K']
iOS Development (New) - https://www.shl.com/solutions/products/product-catalog/view/ios-development-new/ - Yes - No - ['K']
ExpressJS (New) - https://www.shl.com/solutions/products/product-catalog/view/expressjs-new/ - Yes - No - ['K']
Ruby on Rails (New) - https://www.shl.com/solutions/pro

In [23]:
!pip install tabulate


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [25]:
from tabulate import tabulate

results = search_assessments("industrial Engineering")

# Prepare rows for tabulate
table_data = [
    [
        r["name"],
        r["url"],
        r["remote_support"],
        r["adaptive_support"],
        ", ".join(r["test_types"])
    ]
    for r in results
]

# Define headers
headers = ["Assessment Name", "URL", "Remote Support", "Adaptive/IRT", "Test Types"]

# Print table
print(tabulate(table_data, headers=headers, tablefmt="grid"))


+---------------------------------------------------+--------------------------------------------------------------------------------------------------------+------------------+----------------+--------------+
| Assessment Name                                   | URL                                                                                                    | Remote Support   | Adaptive/IRT   | Test Types   |
+===================================================+========================================================================================================+==================+================+==============+
| Industrial Engineering (New)                      | https://www.shl.com/solutions/products/product-catalog/view/industrial-engineering-new/                | Yes              | No             | K            |
+---------------------------------------------------+--------------------------------------------------------------------------------------------------------+--

In [26]:
# Save FAISS index to file
faiss.write_index(index, "shl_faiss_index.index")

# Save metadata (JSON) for later use
with open("shl_metadata.json", "w", encoding="utf-8") as f:
    json.dump(metadata, f, ensure_ascii=False, indent=2)
